# Primal-dual algorithm for vertex cover problem

The basic idea of the algorithm in this notebook is that instead of actually solving the dual LP, we can construct a feasible dual solution with the same properties. In this case, constructing the dual solution is much faster than solving the dual LP, and hence leads to a much faster algorithm.

## The Primal-Dual method for approximation algorithm

the primal-dual algorithm just tries to find a feasible solution and then reduces the duality gap (in a controlled way) 

the general form of primal-dual algorithm is explained as bellow:

assume x is a soulution for primal algorithm and y is a solution for dual algorithm.

<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;padding-left:15px;">
    we pick a feasable y. for example y = 0 <br>
    A = $ \phi $ <br>
    l = 0 : l is a counter <br>
    while A is not feasible in primal :    <br>
    &nbsp;&nbsp;&nbsp;&nbsp; l = l+1 <br>
    &nbsp;&nbsp;&nbsp;&nbsp; $\nu$ = Violation(A) <br>
    &nbsp;&nbsp;&nbsp;&nbsp; Increase $ y_k $ uniformly for all $ T_k \in \nu $ until 
     $ \exists e_l \notin A : \sum_{i:e_l \in T_i} {y_i} = c_i$ <br>
    &nbsp;&nbsp;&nbsp;&nbsp; $ A = A\cup \left \{ e_l \right \} $ <br>
    output(A)
</div>

the general form of algorithm is explaind more [here](http://www.cs.toronto.edu/~bor/2420f10/williamson-primal-dual.pdf)

now we want to solve vertex cover problem by Primal-Dual algorithm

## Vertex cover Problem

Given an undirected graph with vertices $ \in $ V and edges $ \in $ E and edge of $ v_i $ has weight of $ w_i $, the vertex cover problem is to find minimum size vertex cover.

if we illustrate the primal and dual form of this problem, we reach to these two formulas. 

Primal:

$ min \sum_{u}{w_u x_u} $ <br>
$ s.t. $ <br>
$ x_u + x_v \geq 1 \quad \forall uv \in E $ <br>
$  x_u \geq 0 \quad \forall u \in V $

Dual:

$ max \sum_{e}{y_e} $ <br>
$ s.t. $ <br>
$ \sum_{e:u \in e}{y_e} \leq w_u \quad \forall u \in V $ <br>
$  y_e \geq 0 \quad \forall e \in E $

we want to construct integer solution x for (P), y for (D)

we start with $ x = (0 , ... , 0 ) $ and $ y = (0 , ... , 0 ) $

- x has low value but is not feasible
- y is feasible but has low value

now we are trying to make x feasible, step by step. and in order to make it happen, we use y.

the Primal-Dual algorithm for this problem is shown below:

<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;padding-left:15px;">
    start with $ x = (0 , ... , 0 ) $ and $ y = (0 , ... , 0 ) $ <br>
    Repeat : <br>
    &nbsp;&nbsp;&nbsp;&nbsp; pick e = uv such that $ x_u + x_v < 1 $ (a constraint in primal is not satisfied) <br>
    &nbsp;&nbsp;&nbsp;&nbsp; Increase $ y_e $ such that
     $ \sum_{f: u \in f} {y_f} = w_u $ or $ \sum_{f: v \in f} {y_f} = w_v $ <br>
    &nbsp;&nbsp;&nbsp;&nbsp; first case : $ x_u \leftarrow 1 $    &nbsp;&nbsp; second case : $ x_v \leftarrow 1 $ <br>
    output(A)
</div>

during execution of algorithm:
- y remains feasible throughout the execution
- x has fewer and fewer violated constraints

in the end :
- both are feasible



## vertex cover problem in python
### we construct a graph

In [1]:
class Graph:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0 for _ in range(vertices)] for _ in range(vertices)]
        self.weights = [0 for _ in range(vertices)]
        self.E = 0
        self.edges = dict()

    def add_edge(self, u, v):
        self.graph[u][v] = self.graph[v][u] = 1
        self.edges[frozenset((u, v))] = len(self.edges)
        self.E += 1

    def get_edge(self, u, v):
        return self.edges[frozenset([u, v])]


g = Graph(7)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(3, 1)
g.add_edge(2, 3)
g.add_edge(0, 3)
g.add_edge(3, 4)
g.add_edge(0, 4)
g.add_edge(3, 5)
g.add_edge(3, 6)
g.weights = [4, 5, 6, 2, 1, 7, 3]

we want to build this graph

<img src="graph.png" alt="graph">

## we start Primal-Dual algorithm

In [2]:
# 1. initial values of x and y
X = [0 for _ in range(g.V)]
Y = [0 for _ in range(g.E)]


def pick_an_edge(g, X):
    for u, v in g.edges:
        if X[u] + X[v] < 1 and u != v:
            return u, v, g.get_edge(u, v), False
    return -1, -1, -1, True


while True:
    # 2. pick e = uv such that X_u + X_v < 1
    u, v, e, are_all_primal_constraints_satisfied = pick_an_edge(g, X)
    if are_all_primal_constraints_satisfied:
        break
    # 3. increase Y_e such that Sigma Y_f = W_u or Sigma Y_f = W_v
    ΔW_u = g.weights[u] - sum([Y[g.get_edge(u, i)] for i in range(g.V) if g.graph[i][u] == 1])
    ΔW_v = g.weights[v] - sum([Y[g.get_edge(v, i)] for i in range(g.V) if g.graph[i][v] == 1])

    if ΔW_u > ΔW_v:
        Y[g.get_edge(u, v)] += ΔW_v
        X[v] = 1
    else:
        Y[g.get_edge(u, v)] += ΔW_u
        X[u] = 1

In [3]:
print('value of x : ',X)
print('value of y : ',Y)
print('weight : ', sum([X[i] * g.weights[i] for i in range(g.V)]))

value of x :  [1, 1, 0, 1, 0, 0, 0]
value of y :  [4, 1, 0, 2, 0, 0, 0, 0, 0]
weight :  11


- output = {0, 1, 3}
- optimal value = {4, 1, 3}

## faster way
function of picking edges at previous section takes a long time. we can rewrite the algorithm like bellow:
note that we saved edges while we were building the graph. if we didn't do that, we can run a BFS to iterate between edges.

In [4]:
X = [0 for _ in range(g.V)]
Y = [0 for _ in range(g.E)]

for u, v in g.edges:
    e = g.get_edge(u,v)
    ΔW_u = g.weights[u] - sum([Y[g.get_edge(u, i)] for i in range(g.V) if g.graph[i][u] == 1])
    ΔW_v = g.weights[v] - sum([Y[g.get_edge(v, i)] for i in range(g.V) if g.graph[i][v] == 1])
    if ΔW_u > ΔW_v:
        Y[g.get_edge(u, v)] += ΔW_v
        X[v] = 1
    else:
        Y[g.get_edge(u, v)] += ΔW_u
        X[u] = 1
        
print('value of x : ',X)
print('value of y : ',Y)
print('weight : ', sum([X[i] * g.weights[i] for i in range(g.V)]))

value of x :  [1, 1, 0, 1, 0, 0, 0]
value of y :  [4, 1, 0, 2, 0, 0, 0, 0, 0]
weight :  11
